In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-12 22:18:52|MOD:display     |: Model Specifics:
24-05-12 22:18:52|MOD:display     |: Start Process [Data] at Sun May 12 22:18:52 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
Callback : CollectHidden() , load booster data at fit end
{'random_seed': N

24-05-12 22:18:54|MOD:display     |: Finish Process [Data], Cost 1.7 Secs
24-05-12 22:18:54|MOD:display     |: Start Process [Fit] at Sun May 12 22:18:54 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


24-05-12 22:19:00|MOD:__init__    |: Main Process Finished! Cost 7.9 Seconds


(2877, 38, 32)
109326 3104480
[     1      2      4 ... 603997 603998 603999] [20151216 20151223 20151230 20160107 20160114 20160121 20160128 20160204
 20160218 20160225 20160303 20160310 20160317 20160324 20160331 20160408
 20160415 20160422 20160429 20160509 20160516 20160523 20160530 20160606
 20160615 20160622 20160629 20160706 20160713 20160720 20160727 20160803
 20160810 20160817 20160824 20160831 20160907 20160914]


Exception: 

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()